In [1]:
from google.colab import drive
drive.mount ('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/MyDrive/Kaggle Data
!ls

/content/gdrive/MyDrive/Kaggle Data
driver_imgs_list.csv  test  train


In [41]:
!pip install opencv-python==3.4.2.17
!pip install opencv-contrib-python==3.4.2.17

     |████████████████████████████████| 25.0MB 1.9MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


     |████████████████████████████████| 30.6MB 93kB/s 
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


In [12]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab import drive
import matplotlib.pyplot as plt
import joblib
import pickle
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure 
import plotly.express as px

In [13]:

import keras
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img


Resize Images

In [21]:
classes = ['c0', 'c1','c2','c3','c4','c5','c6','c7','c8','c9']
Dict = {'c0' : 0, 'c1' :1, 'c2':2, 'c3':3, 'c4':4,'c5':5,'c6':6,'c7':7,'c8':8,'c9':9}
images = []
Img_labels = []
train_path='/content/gdrive/MyDrive/Kaggle Data/train'

In [22]:
for label in classes:
      path = os.path.join(train_path , label)
      for img in os.listdir(path):
         img = cv2.imread(os.path.join(path,img))
         new_img = cv2.resize(img, (64, 64))
         images.append(new_img)
         Img_labels.append(Dict[label])
         #print(images,Img_labels)
     #print(label)

In [24]:
img=np.array(images)
labels=np.array(Img_labels)
print(img.shape)

(15023, 64, 64, 3)


In [25]:
print(img)

[[[[ 34  43  33]
   [ 40  46  41]
   [ 55  65  56]
   ...
   [192 175  94]
   [220 196 114]
   [252 250 177]]

  [[ 17  15   7]
   [  1  11   5]
   [  1   6   4]
   ...
   [241 224 144]
   [232 218 132]
   [237 224 138]]

  [[ 11  11  12]
   [ 36  58  36]
   [ 76 111  93]
   ...
   [237 222 130]
   [216 199 113]
   [203 193 106]]

  ...

  [[ 49  53  42]
   [ 63  71  48]
   [ 65  69  50]
   ...
   [ 17  18  22]
   [ 11  16  17]
   [ 23  22  24]]

  [[ 63  65  53]
   [ 66  72  49]
   [ 64  68  49]
   ...
   [ 17  19  20]
   [ 17  18  22]
   [ 17  18  22]]

  [[ 67  73  50]
   [ 67  72  51]
   [ 64  68  49]
   ...
   [ 16  15  24]
   [ 16  17  21]
   [ 16  17  21]]]


 [[[ 59  82  71]
   [ 64  87  77]
   [ 68  91  80]
   ...
   [251 231 144]
   [250 244 179]
   [187 170 122]]

  [[193 220 199]
   [228 254 234]
   [229 255 237]
   ...
   [255 245 161]
   [253 244 161]
   [251 247 179]]

  [[224 251 205]
   [195 220 178]
   [ 68  93  46]
   ...
   [252 243 170]
   [248 238 160]
   [255 249

In [26]:

labels

array([0, 0, 0, ..., 9, 9, 9])

### Read Split and Store data

In [27]:
from sklearn.model_selection import train_test_split
x_train,X_test,y_train,Y_test = train_test_split(img,labels, test_size=0.2, random_state=1)
X_train,X_val,Y_train,Y_val = train_test_split(x_train,y_train, test_size=0.1, random_state=1)

In [28]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
print(X_val.shape)
print(Y_val.shape)

(10816, 64, 64, 3)
(10816,)
(3005, 64, 64, 3)
(3005,)
(1202, 64, 64, 3)
(1202,)


## Features Extration

SURF

In [42]:
def get_surf(images, name='surf', save=False):
  # SURF descriptor 
  def get_image_surf(image, vector_size=32):
        alg = cv2.xfeatures2d.SURF_create()
        kps = alg.detect(image, None)
        kps = sorted(kps, key=lambda x: -x.response)[:vector_size]
        
        # Making descriptor of same size
        # Descriptor vector size is 64
        needed_size = (vector_size * 64)
        if len(kps) == 0:
            return np.zeros(needed_size)
        
        kps, dsc = alg.compute(image, kps)
        dsc = dsc.flatten()
        if dsc.size < needed_size:
            # if we have less than 32 descriptors then just adding zeros at the
            # end of our feature vector
            dsc = np.concatenate([dsc, np.zeros(needed_size - dsc.size)])
            
        return dsc
    
  # SURF descriptor for all images
  features = []
  for i, img in enumerate(images):
      dsc = get_image_surf(img)
      features.append(dsc)
  
  result = np.array(features)
  

In [43]:
train_img = get_surf(X_train)
val_img=get_surf(X_val)
test_img= get_surf(X_test)

error: ignored

SIFT


In [33]:
def get_sift(images, name='sift', save=False):
    # SIFT descriptor for 1 image
    def get_image_sift(image, vector_size=32):
        alg = cv2.xfeatures2d.SIFT_create()
        kps = alg.detect(image, None)
        kps = sorted(kps, key=lambda x: -x.response)[:vector_size]
        
        # Making descriptor of same size
        # Descriptor vector size is 128
        needed_size = (vector_size * 64)
        if len(kps) == 0:
            return np.zeros(needed_size)
        
        kps, dsc = alg.compute(image, kps)
        dsc = dsc.flatten()
        if dsc.size < needed_size:
            # if we have less than 32 descriptors then just adding zeros at the
            # end of our feature vector
            dsc = np.concatenate([dsc, np.zeros(needed_size - dsc.size)])
            
        return dsc
    
    # SIFT descriptor for all images
    features = []
    for i, img in enumerate(images):
        dsc = get_image_sift(img)
        features.append(dsc)

    result = np.array(features)

In [37]:
train_img = get_sift(X_train)
val_img=get_sift(X_val)
test_img= get_sift(X_test)

error: ignored